# The Tensor
#### Previously, we made a distinction between vectors (one-dimensional arrays) and matrices (two-dimensional arrays). When we start working with more complicated neural networks, we'll need to use a higher-dimensional arrays as well.

#### In many neural network libraries, *n*-dimensional arrays are referred to as *tensors*, which is what we'll call them too.
#### If I were writing an entire book about deep learning, I'd implement a full-featured `Tensor` class that overloaded Python's arithmetic operators and could handle a variety of other operations. Such an implementation would take a notebook on its own. Here we'll cheat and say that a `Tensor` is just a `list`. This is true in one direction - all our vectors and matrices and higher-dimensional analogues *are* lists. It is certainly not true in the other direction - most Python *lists* are not *n*-dimensional arrays in our sense

#### First, let's write a helper function to find a tensor's *shape*

In [3]:
from typing import List

Tensor = list
def shape(tensor: Tensor) -> List[int]:
    sizes: List[int] = []
    while isinstance(tensor, list):
        sizes.append(len(tensor))
        tensor = tensor[0]
    return sizes

print(shape([1, 2, 3]))
print(shape([[1,2], [3,4], [5,6]]))

[3]
[3, 2]


#### Because tensors can have any number of dimensions, we'll typically need to work with them recursively. We'll do one thing in the one-dimensional case and recurse in the higher-dimensional case:

In [4]:
def is_1d(tensor: Tensor) -> bool:
    """
    If tensor[0] is a list, it's a higher-order tensor. Otherwise, tensor is 1-dimensional (that is, a vector)
    """
    return not isinstance(tensor[0], list)

print(is_1d([1, 2, 3]))

True


#### Which we can use to write a recursive `tensor_sum` function:

In [5]:
def tensor_sum(tensor: Tensor) -> float:
    """ Sums up all the values in the tensor"""
    if is_1d(tensor):
        return sum(tensor)
    else:
        return sum(tensor_sum(tensor_i) for tensor_i in tensor)

print(tensor_sum([1, 2, 3]))

6


#### We'll create a couple of helper functions so that we don't have to rewrite this logic everywhere. The first applies a function elementwise to a single tensor

In [6]:
from typing import Callable

def tensor_apply(f: Callable[[float], float], tensor: Tensor) -> Tensor:
    """ Applies f elementwise""" 
    if is_1d(tensor):
        return[f(x) for x in tensor]
    else:
        return [tensor_apply(f, tensor_i) for tensor_i in tensor]

print(tensor_apply(lambda x: x + 1, [1, 2, 3])) # So in this example, we are adding 1 to every instance of x - each tensor

[2, 3, 4]


#### We can use this to write a function that creates a zero tensor with the same shape as a given tensor:

In [7]:
def zeros_like(tensor: Tensor) -> Tensor:
    return tensor_apply(lambda _: 0.0, tensor)

print(zeros_like([1, 2, 3]))

[0.0, 0.0, 0.0]


#### We'll also need to apply a function to corresponding elements from two tensors (which had better be the exact same shape, although we won't check that)

In [8]:
def tensor_combine(f: Callable[[float, float], float],
                    t1: Tensor,
                    t2: Tensor) -> Tensor:
    """ Applies f to corresponding elements of t1 and t2"""
    if is_1d(t1):
        return [f(x, y) for x,y in zip(t1, t2)]
    else:
        return [tensor_combine(f, t1_i, t2_i) for t1_i, t2_i in zip(t1, t2)]

import operator
print(tensor_combine(operator.add, [1, 2, 3], [4, 5, 6]))
print(tensor_combine(operator.mul, [1, 2, 3], [4, 5, 6]))

[5, 7, 9]
[4, 10, 18]
